In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('survey_final.csv') 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head() 

,Year,Hobbyist,ConvertedComp,Country,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,JobSat,LanguageDesireNextYear,LanguageWorkedWith,OrgSize,UndergradMajor,YearsCodePro
0,2020,Yes,NaN,Germany,Microsoft SQL Server,Elasticsearch;Microsoft SQL Server;Oracle,"Developer, desktop or enterprise applications;...","Master's degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...",Slightly satisfied,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,2 to 9 employees,"Computer science, computer engineering, or sof...",27
1,2020,No,NaN,United Kingdom,NaN,NaN,"Developer, full-stack;Developer, mobile","Bachelor's degree (B.A., B.S., B.Eng., etc.)",Employed full-time,Very dissatisfied,Python;Swift,JavaScript;Swift,"1,000 to 4,999 employees","Computer science, computer engineering, or sof...",4
2,2020,Yes,NaN,Russian Federation,NaN,NaN,NaN,NaN,NaN,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN
3,2020,Yes,NaN,Albania,NaN,NaN,NaN,"Master's degree (M.A., M.S., M.Eng., MBA, etc.)",NaN,Slightly dissatisfied,NaN,NaN,20 to 99 employees,"Computer science, computer engineering, or sof...",4
4,2020,Yes,NaN,United States,MySQL;PostgreSQL,MySQL;PostgreSQL;Redis;SQLite,NaN,"Bachelor's degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,Java;Ruby;Scala,HTML/CSS;Ruby;SQL,NaN,"Computer science, computer engineering, or sof...",8


In [4]:
df.columns

Index(['Year', 'Hobbyist', 'ConvertedComp', 'Country',
       'DatabaseDesireNextYear', 'DatabaseWorkedWith', 'DevType', 'EdLevel',
       'Employment', 'JobSat', 'LanguageDesireNextYear', 'LanguageWorkedWith',
       'OrgSize', 'UndergradMajor', 'YearsCodePro'],
      dtype='object')

In [5]:
print(df.shape)
df.Year.unique() 

(303591, 15)


array([2020, 2019, 2018, 2017], dtype=int64)

#### Checking for missing values

In [6]:
# %age of null values
(df.isnull().sum() / df.shape[0]) * 100

Year                       0.000000
Hobbyist                   0.014823
ConvertedComp             50.205375
Country                    0.307321
DatabaseDesireNextYear    34.066886
DatabaseWorkedWith        28.541360
DevType                   14.711569
EdLevel                    4.504415
Employment                 1.924629
JobSat                    25.612419
LanguageDesireNextYear    18.288092
LanguageWorkedWith        15.329506
OrgSize                   25.367353
UndergradMajor            18.151065
YearsCodePro              21.119862
dtype: float64

In [9]:
#  taking missing data for salary separately
# %age of missing values(for salary) contributed by each year
temp = df[df.ConvertedComp.isnull()==True]
(temp.Year.value_counts().head(10) / temp.shape[0]) * 100 

2018    33.560777
2017    25.259974
2019    21.690209
2020    19.489040
Name: Year, dtype: float64

Fixing Hobbyist column

In [11]:
# del temp
df.groupby('Year').Hobbyist.unique().values

array([array(['Yes, both', 'Yes, I program as a hobby', 'No',
       'Yes, I contribute to open source projects'], dtype=object),
       array(['Yes', 'No'], dtype=object),
       array(['Yes', 'No'], dtype=object),
       array(['Yes', 'No', nan], dtype=object)], dtype=object)

Hobby is either Yes/No is all years except in 2017, where it is either Yes/No/yes_as_a_hobby/yes_as_OSC/both so we'll format them properly

In [12]:
df.Hobbyist.replace(['Yes, both','Yes, I program as a hobby',
                      'Yes, I contribute to open source projects'],'Yes',inplace=True)

In [13]:
# dict.fromkeys(['Beer','Alcohol','Beverage','Drink'], 'Drink') # alternative

Fixing Database Desire column

In [14]:
from collections import Counter
db_desire = Counter()
for ele in df.DatabaseDesireNextYear.dropna():
    t = ele.strip().split(';')
    db_desire.update(t) 
    
    
# to handle duplicates
for keys in list(db_desire.keys()):  # as you can't iterate over a dict while also modifying it
    if keys.startswith(" "):
        db_desire[keys.strip()] = db_desire[keys.strip()] + db_desire[keys]
        db_desire.pop(keys) # removing after updating
        
        

# for 2 special cases
db_desire['Microsoft SQL Server'] = db_desire['Microsoft SQL Server'] + db_desire['SQL Server']
db_desire.pop('SQL Server')
db_desire['Amazon DynamoDB'] = db_desire['Amazon DynamoDB'] + db_desire['DynamoDB']
db_desire.pop('DynamoDB')
# final
db_desire

Counter({'Microsoft SQL Server': 53706,
         'MySQL': 79824,
         'PostgreSQL': 71955,
         'MongoDB': 66809,
         'Firebase': 20521,
         'Cassandra': 16969,
         'Elasticsearch': 39767,
         'MariaDB': 23240,
         'Redis': 48221,
         'SQLite': 44977,
         'Oracle': 21318,
         'Couchbase': 4147,
         'IBM DB2': 935,
         'Other(s):': 4516,
         'Amazon RDS/Aurora': 4630,
         'Microsoft Azure (Tables, CosmosDB, SQL, etc)': 8325,
         'IBM Db2': 905,
         'Amazon DynamoDB': 16960,
         'Apache Hive': 2653,
         'Neo4j': 3790,
         'Apache HBase': 2312,
         'Google BigQuery': 5036,
         'Memcached': 3587,
         'Google Cloud Storage': 7515,
         'Amazon Redshift': 3183})

Fixing Database-Worked-with column

In [15]:
db_worked = Counter()
for ele in df.DatabaseWorkedWith.dropna():
    t = ele.strip().split(';')
    db_worked.update(t) 

    
# to handle duplicates
for keys in list(db_worked.keys()):  # as you can't iterate over a dict while also modifying it
    if keys.startswith(" "):
        db_worked[keys.strip()] = db_worked[keys.strip()] + db_worked[keys]
        db_worked.pop(keys) # removing after updating
        
    
# for 2 special cases
db_worked['Microsoft SQL Server'] = db_worked['Microsoft SQL Server'] + db_worked['SQL Server']
db_worked.pop('SQL Server')
db_worked['Amazon DynamoDB'] = db_worked['Amazon DynamoDB'] + db_worked['DynamoDB']
db_worked.pop('DynamoDB')
# final
db_worked

Counter({'Elasticsearch': 26849,
         'Microsoft SQL Server': 75572,
         'Oracle': 30918,
         'MySQL': 116882,
         'PostgreSQL': 73547,
         'Redis': 41152,
         'SQLite': 57981,
         'MariaDB': 29566,
         'Firebase': 16740,
         'MongoDB': 59192,
         'IBM DB2': 1421,
         'Cassandra': 9846,
         'Couchbase': 2454,
         'Other(s):': 5059,
         'Amazon RDS/Aurora': 3388,
         'Microsoft Azure (Tables, CosmosDB, SQL, etc)': 5203,
         'Memcached': 3615,
         'IBM Db2': 1677,
         'Google Cloud Storage': 3634,
         'Amazon DynamoDB': 11580,
         'Apache HBase': 1107,
         'Apache Hive': 1438,
         'Amazon Redshift': 1464,
         'Google BigQuery': 1384,
         'Neo4j': 1572})

Combining Both

In [16]:
t1 = pd.DataFrame(pd.Series(db_desire).reset_index())
t2 = pd.DataFrame(pd.Series(db_worked).reset_index()) 
total_table_db = t1.merge(t2,suffixes=('_Desired','_Worked_with'),how='outer',on='index')
total_table_db.columns = ['DataBase','Desired to use','Already using']
# total_table_db.to_csv('4yrs_db.csv',index=False) 

Fixing Developer Type Column

In [17]:
devs = Counter()
for ele in df.DevType.dropna():
    t = [i.strip() for i in ele.strip().split(';')]  # another simpler way
    devs.update(t) 

    
devs['Developer, front-end'] = devs['Developer, front-end'] + devs['Front-end developer']
devs.pop('Front-end developer') 

devs['Developer, back-end'] = devs['Developer, back-end'] + devs['Back-end developer']
devs.pop('Back-end developer')

devs['Developer, full-stack'] = devs['Developer, full-stack'] + devs['Full-stack developer'] + devs['Web developer']
devs.pop('Web developer')
devs.pop('Full-stack developer') 

devs['Data scientist or machine learning specialist'] += devs['Data scientist'] + devs['Machine learning specialist']    
devs.pop('Machine learning specialist')
devs.pop('Data scientist')

devs['System administrator'] = devs['System administrator'] + devs['Systems administrator']
devs.pop('Systems administrator');

In [18]:
# # outputting to a csv file w/o pandas
# import csv
# # to be ran once
# with open('4yrs_devs.csv','w') as csvfile:
#     writer=csv.writer(csvfile)
#     for key, value in devs.items():
#         writer.writerow([key]+[value])
        

Fixing EdLevel column

In [19]:
df.EdLevel.unique()

array(["Master's degree (M.A., M.S., M.Eng., MBA, etc.)",
       "Bachelor's degree (B.A., B.S., B.Eng., etc.)", nan,
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       'Professional degree (JD, MD, etc.)',
       'Some college/university study without earning a degree',
       'Associate degree (A.A., A.S., etc.)',
       'Other doctoral degree (Ph.D., Ed.D., etc.)',
       'Primary/elementary school',
       'I never completed any formal education',
       "Bachelor's degree (BA, BS, B.Eng., etc.)",
       "Master's degree (MA, MS, M.Eng., MBA, etc.)",
       'Other doctoral degree (Ph.D, Ed.D., etc.)', 'Associate degree',
       "Bachelor‚'s degree (BA, BS, B.Eng., etc.)",
       'Full-stack developer',
       "Master‚'s degree (MA, MS, M.Eng., MBA, etc.)", 'Secondary school',
       "Some college/university study without earning a bachelor's degree",
       "Bachelor's degree", 'Doctoral degree', "Master's degree",
       'Profession

In [20]:
b = dict.fromkeys(["Bachelor's degree (B.A., B.S., B.Eng., etc.)",
              "Bachelor‚'s degree (BA, BS, B.Eng., etc.)",
              "Bachelor's degree (BA, BS, B.Eng., etc.)","Bachelor's degree"],"Some Bachelor's Degree") 

m = dict.fromkeys(
["Master's degree (M.A., M.S., M.Eng., MBA, etc.)","Master's degree (MA, MS, M.Eng., MBA, etc.)",
 "Master‚'s degree (MA, MS, M.Eng., MBA, etc.)","Master's degree"],"Some Master's Degree"
)

s = dict.fromkeys(
['Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
'Primary/elementary school','Secondary school'],'Some School'
)

p = dict.fromkeys(
['Professional degree (JD, MD, etc.)','Some college/university study without earning a degree',
'Full-stack developer',"Some college/university study without earning a bachelor's degree",
'Professional degree'],'Some Professional degree'
)

d = dict.fromkeys(
['Other doctoral degree (Ph.D., Ed.D., etc.)', 'Other doctoral degree (Ph.D, Ed.D., etc.)',
'Doctoral degree'],'Some Doctoral degree'
)

a = dict.fromkeys(
['Associate degree (A.A., A.S., etc.)','Associate degree'],'Some Associate degree'
)

final_changes = {**b,**m,**s,**p,**d,**a} 

In [21]:
# finally replacing nulls with 'I prefer not to answer' 
df.EdLevel = df.EdLevel.replace(final_changes).fillna('I prefer not to answer')

In [22]:
df.EdLevel.value_counts() 

Some Bachelor's Degree                    130943
Some Master's Degree                       65218
Some Professional degree                   41741
Some School                                33338
I prefer not to answer                     14784
Some Associate degree                       7751
Some Doctoral degree                        7644
I never completed any formal education      2172
Name: EdLevel, dtype: int64

Fixing JobSat column

In [23]:
final_changes = {
    'Extremely satisfied':'Very satisfied',
    'Extremely dissatisfied':'Very dissatisfied',
    'Moderately satisfied':'Slightly satisfied',
    'Moderately dissatisfied':'Slightly dissatisfied',
    '0.0':'Very dissatisfied',
    '1.0':'Very dissatisfied',
    '2.0':'Very dissatisfied',
    '3.0':'Slightly dissatisfied',
    '4.0':'Slightly dissatisfied',
    '5.0':'Neither satisfied nor dissatisfied',  
    '6.0':'Slightly satisfied', 
    '7.0':'Slightly satisfied',  
    '8.0':'Slightly satisfied',  
    '9.0':'Very satisfied', 
    '10.0':'Very satisfied'
    
} 

In [24]:
df.JobSat.replace(final_changes,inplace=True)

In [25]:
def custom(row):
#     print(row)
    if type(row)==np.float:
        if row >= 8.0:
            return 'Very satisfied'
        elif row >= 6.0:
            return 'Slightly satisfied'
        elif row >= 4.0:
            return 'Neither satisfied nor dissatisfied'
        elif row >= 2.0:
            return 'Slightly dissatisfied'
        elif row >= 0.0:
            return 'Very dissatisfied'
    else:
        return row

In [26]:
df.JobSat = df.JobSat.apply(custom)

Fixing Organization size Column

In [27]:
df.OrgSize.unique()
# len(df.OrgSize.isna()==True)

array(['2 to 9 employees', '1,000 to 4,999 employees', nan,
       '20 to 99 employees',
       'Just me - I am a freelancer, sole proprietor, etc.',
       '10,000 or more employees', '100 to 499 employees',
       '500 to 999 employees', '10 to 19 employees',
       '5,000 to 9,999 employees', '2-9 employees',
       'Fewer than 10 employees', "I don't know",
       'I prefer not to answer'], dtype=object)

In [28]:
final_changes = {
    '2-9 employees':'2 to 9 employees',
    'Fewer than 10 employees':'2 to 9 employees',
    
}

In [29]:
df.OrgSize = df.OrgSize.replace(final_changes).fillna("I don't know") 

In [30]:
df.OrgSize.unique() 

array(['2 to 9 employees', '1,000 to 4,999 employees', "I don't know",
       '20 to 99 employees',
       'Just me - I am a freelancer, sole proprietor, etc.',
       '10,000 or more employees', '100 to 499 employees',
       '500 to 999 employees', '10 to 19 employees',
       '5,000 to 9,999 employees', 'I prefer not to answer'], dtype=object)

Fixing UndergradMajor Column

In [31]:
df.UndergradMajor.unique()

array(['Computer science, computer engineering, or software engineering',
       nan, 'Mathematics or statistics',
       'Another engineering discipline (such as civil, electrical, mechanical, etc.)',
       'A humanities discipline (such as literature, history, philosophy, etc.)',
       'A health science (such as nursing, pharmacy, radiology, etc.)',
       'Information systems, information technology, or system administration',
       'Web development or web design',
       'A natural science (such as biology, chemistry, physics, etc.)',
       'Fine arts or performing arts (such as graphic design, music, studio art, etc.)',
       'I never declared a major',
       'A social science (such as anthropology, psychology, political science, etc.)',
       'A business discipline (such as accounting, finance, marketing, etc.)',
       'Another engineering discipline (ex. civil, electrical, mechanical)',
       'A business discipline (ex. accounting, finance, marketing)',
       'A natura

In [32]:
temp = dict.fromkeys(['Computer science or software engineering' , 
                   'Computer engineering or electrical/electronics engineering',
                     'Computer programming or Web development'],
                  'Computer science, computer engineering, or software engineering')
final_changes = {
    'Another engineering discipline (such as civil, electrical, mechanical, etc.)':'A non-computer-focused engineering discipline',                                                   
    'Another engineering discipline (ex. civil, electrical, mechanical)':'A non-computer-focused engineering discipline',
    'A natural science':'A natural science (such as biology, chemistry, physics, etc.)',
    'A business discipline':'A business discipline (such as accounting, finance, marketing, etc.)',
    'A business discipline (ex. accounting, finance, marketing)':'A business discipline (such as accounting, finance, marketing, etc.)',
    'Fine arts or performing arts':'Fine arts or performing arts (such as graphic design, music, studio art, etc.)',
    'A humanities discipline':'A humanities discipline (ex. literature, history, philosophy)',
    'A health science':'A health science (ex. nursing, pharmacy, radiology)',
    'A natural science (ex. biology, chemistry, physics)':'A natural science (such as biology, chemistry, physics, etc.)',
    'A health science (such as nursing, pharmacy, radiology, etc.)':'A health science (ex. nursing, pharmacy, radiology)',
    'A social science':'A social science (such as anthropology, psychology, political science, etc.)',
    'Fine arts or performing arts (ex. graphic design, music, studio art)':'Fine arts or performing arts (such as graphic design, music, studio art, etc.)',
    'A social science (ex. anthropology, psychology, political science)':'A social science (such as anthropology, psychology, political science, etc.)'
}
final_changes = {**temp,**final_changes}
final_changes

{'Computer science or software engineering': 'Computer science, computer engineering, or software engineering',
 'Computer engineering or electrical/electronics engineering': 'Computer science, computer engineering, or software engineering',
 'Computer programming or Web development': 'Computer science, computer engineering, or software engineering',
 'Another engineering discipline (such as civil, electrical, mechanical, etc.)': 'A non-computer-focused engineering discipline',
 'Another engineering discipline (ex. civil, electrical, mechanical)': 'A non-computer-focused engineering discipline',
 'A natural science': 'A natural science (such as biology, chemistry, physics, etc.)',
 'A business discipline': 'A business discipline (such as accounting, finance, marketing, etc.)',
 'A business discipline (ex. accounting, finance, marketing)': 'A business discipline (such as accounting, finance, marketing, etc.)',
 'Fine arts or performing arts': 'Fine arts or performing arts (such as graph

In [33]:
df.UndergradMajor = df.UndergradMajor.replace(final_changes)

Fixing LanguageWorkedWith

In [34]:
lang_worked = Counter()
for ele in df.LanguageWorkedWith.dropna():
    t = [i.strip() for i in ele.strip().split(';')]
    lang_worked.update(t) 
    
# REMOVE DUPLICATES

lang_worked['Bash/Shell/PowerShell'] += lang_worked['Bash/Shell']
lang_worked.pop('Bash/Shell')

lang_worked['HTML/CSS'] += lang_worked['HTML'] + lang_worked['CSS']
lang_worked.pop('HTML')
lang_worked.pop('CSS')

lang_worked

Counter({'C#': 82279,
         'HTML/CSS': 196254,
         'JavaScript': 169054,
         'Swift': 19981,
         'Objective-C': 13885,
         'Python': 104659,
         'Ruby': 23254,
         'SQL': 134844,
         'Java': 104500,
         'PHP': 67473,
         'C': 53387,
         'TypeScript': 52419,
         'Bash/Shell/PowerShell': 82143,
         'Kotlin': 13596,
         'R': 15777,
         'VBA': 12535,
         'Perl': 6165,
         'Scala': 11903,
         'C++': 61750,
         'Go': 23541,
         'Haskell': 5489,
         'Rust': 10298,
         'Dart': 4459,
         'Julia': 1464,
         'Assembly': 17069,
         'WebAssembly': 1015,
         'Other(s):': 7920,
         'Elixir': 2825,
         'Erlang': 2716,
         'Clojure': 3594,
         'F#': 4122,
         'Matlab': 5495,
         'CoffeeScript': 3635,
         'Lua': 3850,
         'VB.NET': 6058,
         'Groovy': 4239,
         'Cobol': 590,
         'Visual Basic 6': 3382,
         'Delphi/Obj

Fixing LanguageDesireNextYear

In [35]:
lang_desire = Counter()
for ele in df.LanguageDesireNextYear.dropna():
    t = [i.strip() for i in ele.strip().split(';')]
    lang_desire.update(t)

# REMOVE DUPLICATES

lang_desire['Bash/Shell/PowerShell'] += lang_desire['Bash/Shell']
lang_desire.pop('Bash/Shell')

lang_desire['HTML/CSS'] += lang_desire['HTML'] + lang_desire['CSS']
lang_desire.pop('HTML')
lang_desire.pop('CSS')

lang_desire

Counter({'C#': 69018,
         'HTML/CSS': 119404,
         'JavaScript': 132267,
         'Python': 111187,
         'Swift': 26519,
         'Objective-C': 10274,
         'Java': 73852,
         'Ruby': 22583,
         'Scala': 16555,
         'PHP': 42477,
         'Rust': 29333,
         'R': 19680,
         'SQL': 100301,
         'Go': 46751,
         'TypeScript': 61254,
         'Assembly': 13116,
         'Bash/Shell/PowerShell': 53782,
         'C': 34524,
         'Haskell': 8762,
         'Perl': 4664,
         'C++': 50056,
         'Julia': 3153,
         'Kotlin': 34828,
         'Dart': 9316,
         'VBA': 5337,
         'Elixir': 4168,
         'WebAssembly': 8440,
         'Erlang': 5165,
         'Other(s):': 7912,
         'Clojure': 5853,
         'F#': 7730,
         'Matlab': 3674,
         'Ocaml': 1143,
         'CoffeeScript': 3048,
         'Delphi/Object Pascal': 1026,
         'Groovy': 3238,
         'Hack': 837,
         'VB.NET': 3819,
         'Lua':

Combining both

In [36]:
t1 = pd.DataFrame(pd.Series(lang_desire).reset_index())
t2 = pd.DataFrame(pd.Series(lang_worked).reset_index()) 
total_table_lang = t1.merge(t2,suffixes=('_Desired','_Worked_with'),how='outer',on='index')
total_table_lang.columns = ['Language','Desired to use','Already using']
# total_table_lang.to_csv('4yrs_lang.csv',index=False) 

### For Year - Wise analysis of Languages & DataBases

In [37]:
def create(column=None,col_names=None):
    if column==None:
        return "Please Enter the column"
    if col_names==None:
        col_names = ['Tool','Count','Year']
    def custom2(column):
        for ele in range(2017,2021,1):
            temp = df[df.Year==ele]
            cnter = Counter()
            for ele in temp[f'{column}'].dropna():
                t = [i.strip() for i in ele.strip().split(';')]
                cnter.update(t)
            yield cnter

    year_wise = list(custom2(column)) 
    df_ = pd.DataFrame(columns=['index',0,'YEAR'])

    k = 2017
    for ele in year_wise:
        temp = pd.DataFrame(pd.Series(ele).reset_index())
        temp['YEAR'] = k
        df_ = pd.concat([df_,temp])
        k += 1

    df_.columns = col_names
    
    return df_




In [38]:
# for languages
t1 = create(column = 'LanguageDesireNextYear',col_names=['Desired Language','Count','Year']) 
t2 = create(column = 'LanguageWorkedWith',col_names=['Language Worked_with','Count','Year']) 

final_table = t1.merge(t2,suffixes=('_Desired','_Worked_with'),how='outer',
                        left_on = ['Desired Language','Year'],
         right_on=['Language Worked_with','Year']) 

final_table.drop("Language Worked_with",axis=1,inplace=True)

final_table.rename(columns={'Desired Language':'Language'},inplace=True)

final_table.to_csv('Language.csv',index=False) 

In [39]:
# for databases
t1 = create(column = 'DatabaseDesireNextYear',col_names=['Desired DataBase','Count','Year']) 
t2 = create(column = 'DatabaseWorkedWith',col_names=['DataBase Worked_with','Count','Year'])

final_table = t1.merge(t2,suffixes=('_Desired','_Worked_with'),how='outer',
                        left_on = ['Desired DataBase','Year'],
         right_on=['DataBase Worked_with','Year']) 

final_table.drop("DataBase Worked_with",axis=1,inplace=True)

final_table.rename(columns={'Desired DataBase':'DataBase'},inplace=True)

# final_table.to_csv('DataBases.csv',index=False) 

### Imputing Missing Values for Salary

In [44]:
# searched for patterns in missing values(for salaries) using multiple variables but didn't find any
# for imputation we'll use group median imputation (grp by year) , as data is highly skewed for all years 
# so we can't mean impute.



In [60]:
med_17 = df.query("Year==2017").ConvertedComp.median()
med_18 = df.query("Year==2018").ConvertedComp.median()
med_19 = df.query("Year==2019").ConvertedComp.median()
med_20 = df.query("Year==2020").ConvertedComp.median()


def imputer(row):
    year = row[0]
    val = row[1]
    if pd.isnull(val):
        if year==2017:
            return med_17
        elif year==2018:
            return med_18
        elif year==2019:
            return med_19
        elif year==2020:
            return med_20
    else:
        return val
    
df.ConvertedComp = df[['Year','ConvertedComp']].apply(imputer,axis=1)
        

In [61]:
# prepared data for visualization

# df.to_csv('.\Data to Visualize\Final Data.csv',index=False) 

## End 